# Convex optimization

### Definition: 
$f$ is **convex** if for all $\theta \in [0,1]$
$$
f(\theta x + (1-\theta)y ) \leq \theta f(x) + (1-\theta) f(y)
$$

equivalently, 

* $f$ has nonnegative (upward) curvature
* the graph of $f$ never lies above its chords
* $f'' \geq 0$ (if $f$ is differentiable)
* epigraph $\{(x,t): f(x) \leq t\}$ is a convex set
    * implies sublevel sets $\{x: f(x) \leq \alpha\}$ are convex sets

<img src="figures/chord.png">

### Definition
* $f$ is **concave** iff $-f$ is convex
* $f$ is **affine** iff $f$ is both convex and concave

## Convex optimization (traditional form)

$$
\begin{array}{ll} 
\mbox{minimize}  & f_0(x) \\
\mbox{subject to} & f_i(x) \leq 0, \quad i=1, \ldots, m_1\\
& h_i(x) = 0, \quad i=1, \ldots, m_2\\
\end{array}
$$

* variable $x\in \mathbf{R}^n$
* $f_i$ are all convex
* $h_i$ are all affine

## Convex optimization (conic form)

$$
\begin{array}{ll} 
\mbox{minimize}  & c^T x \\
    \mbox{subject to} & Ax = b\\
    & x \in \mathcal K\\
\end{array}
$$

where $\mathcal K$ is a **convex cone**

$$ x \in \mathcal K \iff rx \in \mathcal K, \quad \forall r>0$$

examples:

* positive orthant 

$$\mathcal K_+ = \{x: x\geq 0\}$$
    
* second order cone 

$$\mathcal K_{\mathrm{SOC}} = \{(x,t): \|x\|_2 \leq t\}$$
    
* semidefinite cone 

$$\mathcal K_{\mathrm{SDP}} = \{X: X = X^T,~ v^T X v \geq 0,~ \forall v \in \mathbf{R}^n\}$$
    

# Quick convex prototyping

In [1]:
using Convex

INFO: Recompiling stale cache file /Users/madeleine/.julia/lib/v0.5/Convex.ji for module Convex.


## Variables

In [2]:
# Scalar variable
x = Variable()

Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [3]:
# (Column) vector variable
y = Variable(4)

Variable of
size: (4, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [4]:
# Matrix variable
z = Variable(4, 2)

Variable of
size: (4, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [5]:
# Complex variable
c = ComplexVariable(5)

Variable of
size: (5, 1)
sign: Convex.ComplexSign()
vexity: Convex.AffineVexity()

# Expressions

* functions operate on variables to form *convex expressions*

In [6]:
# indexing, multiplication, addition
e1 = y[1] + 2*x

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [7]:
# expressions can be affine, convex, or concave
e3 = sqrt(x)

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()


In [10]:
# many built-in functions for Convex variables
e2 = 4 * pos(x) + maximum(abs(y)) + norm(z[:,1],2)

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [11]:
# not convex! more on this later...
e3 = square(y[1]) + sqrt(x)

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.NotDcp()


# Constraints

In [12]:
# affine equality constraint
A = randn(3,4); b = randn(3)
c1 = A*y == b

Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [-2.23407,0.0315179,1.00965]
vexity: Convex.AffineVexity()

In [13]:
# convex inequality constraint
c2 = norm(y,2) <= 2

Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()

In [14]:
# constraints are parsed to verify convexity
c3 = norm(z[:,1]) + 7 <= min(y[3],x)

Constraint:
<= constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: AbstractExpr with
head: min
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConcaveVexity()

vexity: Convex.ConvexVexity()

In [15]:
# not convex!
c3 = norm(z[:,1]) + 7 <= max(y[3],x)

Constraint:
<= constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: AbstractExpr with
head: max
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

vexity: Convex.NotDcp()

# Problems

In [17]:
objective = 2*x + 1 - sqrt(sum(y))
constraint = x >= maximum(y)
p = minimize(objective, constraint)

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
>= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
rhs: AbstractExpr with
head: maximum
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

vexity: Convex.ConvexVexity()
current status: not yet solved

In [18]:
# solve the problem
solve!(p)
p

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 26
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 8, constraints m = 11
Cones:	primal zero / dual free vars: 1
	linear vars: 7
	soc vars: 3, soc blks: 1
Setup time: 7.86e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  4.73e-04 
   100| 1.51e-05  2.56e-04  1.47e-04  5.00e-01  5.00e-01  8.07e-17  5.44e-04 
   120| 3.77e-06  6.38e-05  3.67e-05  5.00e-01  5.00e-01  0.00e+00  5.60e-04 
----------------------------------

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
>= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
value: 0.2499283491590887
rhs: AbstractExpr with
head: maximum
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

vexity: Convex.ConvexVexity()
current status: Optimal

In [19]:
p.optval

0.5000079118188752

In [21]:
# can evaluate expressions directly
evaluate(objective)

1×1 Array{Float64,2}:
 0.500004

In [22]:
x.value

0.2499283491590887

In [26]:
# lagrange dual variable
constraint.dual

2.0000000010988517

## Problem variants

* maximization problems 
    ```julia
    maximize(concaveExpr, constraints)
    ```
* constraint satisfaction problems 
    ```julia
    satisfy(constraints)
    ```

## Many more examples Convex in action [on GitHub](https://github.com/JuliaOpt/Convex.jl/tree/master/examples)

# Disciplined convex programming (DCP)

Infer convexity of expressions by induction:

* variables have known vexity (affine) and sign
* library of atoms with known vexity and sign (as function of their arguments)

More information at [dcp.stanford.edu](dcp.stanford.edu)

## Convexity inference
1. $f \circ g(x)$ is convex in $x$ if
    * $f$ is convex increasing and $g$ is convex 
    * $f$ is convex decreasing and $g$ is concave 

1. $f \circ g(x)$ is concave in $x$ if
    * $f$ is concave increasing and $g$ is concave 
    * $f$ is concave decreasing and $g$ is convex

For smooth functions, derivation via chain rule:
$$
(f \circ g)''(x) = f''(g(x))(g(x))^2 + f'(g(x))g''(x)
$$

example: 
* `+` is convex and increasing in its arguments
* so `convexExpr + convexExpr` is convex

# Library of atoms

See [the docs](convexjl.readthedocs.io/operations.html) for the full list of operations

## Convex expressions to convex optimization

Convex simply checks that 

* objective is
    * `minimize(convexExpr)`
    * `maximize(concaveExpr)`
* constraints are
    * `convexExpr <= concaveExpr`
    * `affineExpr == 0`
* (remember affine expressions are both convex and concave)

## DCP examples

In [27]:
# $f$ is convex increasing and $g$ is convex
square(pos(x))

AbstractExpr with
head: qol_elem
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [29]:
# $f$ is convex decreasing and $g$ is concave 
invpos(sqrt(x))

AbstractExpr with
head: qol_elem
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [30]:
# $f$ is concave increasing and $g$ is concave 
sqrt(sqrt(x))

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()


In [31]:
# not every convex expression is DCP compliant
sqrt(square(x))
# here, better use abs(x)

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.NotDcp()


In [32]:
# convex + concave => not DCP compliant
e3 = square(y[1]) + sqrt(x)

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.NotDcp()


# How does it work?

Convex solves problems by:
* constructing abstract syntax tree
* parsing to conic form
* passing to solver

## Construct abstract syntax tree

(done automatically during parsing)

* for objective
* for each constraint

example:

    p = minimize(norm(A*x-b,2), x>=0)
    
becomes
    
    p.objective   = (:norm_2, [(:-, [(:*, [A, x]), b])])
    p.constraints = [(:<=, 0, x)]
    
**Remark:** AST does not depend on solver format (or on convexity of expressions)

# Conic Form

Convex automatically reformulates problems into conic form to pass to conic form solvers.

## Parse to conic form: introduce new variables

one variable per atom

$$
\begin{array}{ll}
\mbox{minimize} & v_3 \\
\mbox{subject to} & v_1 = Ax \\
& v_2 = v_1 - b \\
& v_3 = \|v_2\|_2 \\
& x\geq 0
\end{array}
$$

## Parse to conic form: relax convex constraints

nonconvex equality constraints become convex inequality constraints

$$
\begin{array}{ll}
\mbox{minimize} & v_3 \\
\mbox{subject to} & v_1 = Ax \\
& v_2 = v_1 - b \\
& v_3 \geq \|v_2\|_2 \\
& x\geq 0
\end{array}
$$

**Convexity:** relaxation is tight at solution

## Parse to conic form:

conic form is defined recursively:

$$
\begin{array}{ll}
\mbox{minimize} & v_3 \\
\mbox{subject to} & v_1 - Ax = 0 \\
& v_1 - v_2 = b \\
& (v_2, v_3) \in \mathcal K_{\mathrm{SOC}}^m \\
& x \in \mathcal K_+^n
\end{array}
$$

In [39]:
## In code
m,n = 4,2
A = randn(m,n)
b = randn(m)
x = Variable(n)
conic_form!((norm(A*x-b,2))) # gives coefficients of variables in objective and in constraint, in sparse form

DataStructures.OrderedDict{UInt64,Tuple{Union{AbstractArray,Number},Union{AbstractArray,Number}}} with 2 entries:
  0xcf91c72d1de502cf => (…
  0xda59128f4ac0b6bc => (…

## Pass to MathProgBase: construct matrices

$$
\begin{array}{ll}
\mbox{minimize} & 
    \begin{bmatrix}
    0 & 0 & 1 & 0
    \end{bmatrix}
    \begin{bmatrix}
    v_1 \\ v_2 \\ v_3 \\ x
    \end{bmatrix} \\
\mbox{subject to} & 
    \begin{bmatrix}
    I & 0 & 0 & -A \\
    I & -I & 0 & 0
    \end{bmatrix}
    \begin{bmatrix}
    v_1 \\ v_2 \\ v_3 \\ x
    \end{bmatrix}
    =  
    \begin{bmatrix}
    0 \\ b
    \end{bmatrix} \\
& 
    \begin{bmatrix}
    v_1 \\ v_2 \\ v_3 \\ x
    \end{bmatrix}
    \in \mathbf{R}^m \times \mathcal K_{\mathrm{SOC}}^m \times \mathcal K_+^n
\end{array}
$$

* we have arrived at conic form!

## Pass to solver

problem is now in standard conic form
$$
\begin{array}{ll} 
\mbox{minimize}  & c^T x \\
    \mbox{subject to} & Ax = b\\
    & x \in \mathcal K\\
\end{array}
$$

### Call solver libraries

* $\mathcal K = $ positive orthant $\implies$ linear program
* $\mathcal K = $ second order cone $\implies$ SOCP
* $\mathcal K = $ semidefinite cone $\implies$ SDP
* $\mathcal K = $ exponential cone $\implies$ exponential cone program

### Lots of solvers to choose from!

Can use any solver from [MathProgBase](https://github.com/JuliaOpt/MathProgBase.jl) that supports the cones you need